# Red de Bengio
Se pretende predecir el texto que le sigue a cierta oración utilizando una red de Bengio

In [ ]:
#Autores: Daniel Castillo, Karla Salas con ayuda del profesor Mijangos
#Para ver las palabras
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
# nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords #Listas de stopwords
from nltk.tokenize import word_tokenize #Tokens
import re #regex
import numpy as np
from operator import itemgetter
import pickle # Guardar objetos
from tqdm import tqdm #Medir el progreso del entrenamiento

Clase Bengio: para la red neuronal

Retomamos lo d ela práctica 2 a manera de comparar este modelo con otros de los que se hablará más adelante

In [ ]:
class Bengio:
    '''
    Aplica la arquitectura de Bengios
    
    Args:
        bigrams (list): Lista de bigramas por oración en el corpus
        voc (dic): Diccionario de palabras con su índice asociado
        dim (int): Unidades ocultas
        nn_hdim (int): Unidades en la segunda capa
    '''
    def __init__(self, bigrams, voc, dim, nn_hdim):
        np.random.seed(0)
        self.bigrams = bigrams
        self.voc = voc
        # unidades de la capa oculta
        self.dim = dim
        # unidades de la segunda capa
        self.nn_hdim = nn_hdim
        N = len(voc)
        #Embedding (este vector se guarda para la siguiente tarea)
        self.C = np.random.randn(dim, N) / np.sqrt(N)
        #U (a |V | × h matrix) - hidden-to-output weights
        self.U = np.random.randn(nn_hdim, dim) / np.sqrt(dim)
        self.b = np.zeros((1, self.nn_hdim)) #bias
        # W (a |V | × (n − 1)m matrix) word features to output weights
        self.W = np.random.randn(N, nn_hdim) / np.sqrt(nn_hdim)
        self.c = np.zeros((1, N))
    
    def train(self, its, eta):
        '''
        Entrena la red de Bengios
        Obtiene la probabilidad de transitar de una palabra a otra

        Args:
            its (int): Iteraciones
            eta (int): radio de aprendizaje
        '''
        for i in tqdm(range(0,its)):
            for ex in self.bigrams:
                #Forward
                f, a = self.forward(ex[0])
                #Backward, pasos descritos en el paper
                #Variable de salida, (a).1
                d_out = f
                d_out[ex[1]] -= 1
                #Variable para la capa oculta
                d_tanh = (1-a**2)*np.dot(self.W.T,d_out)
                #Variable de embedding
                d_emb = np.dot(self.U.T, d_tanh)
                #Actualizacion de salida
                self.W -= eta*np.outer(d_out,a)
                #Actualiza bias de salida
                self.c -= eta*d_out #[j]
                #Actualizacion de capa oculta
                self.U -= eta*np.outer(d_tanh,self.C.T[ex[0]])
                #Actualiza bias
                self.b -= eta*d_tanh
                #Actualizacion de embedding
                self.C.T[ex[0]] -= eta*d_emb

    def forward(self, x): 
        '''
        Etapa forward de la red 
        sirve para entrenar y evaluar el modelo

        Args:
            x (str): Palabra a calcular la probabilidad dado un contexto
        '''
        x = self.voc['<oov>' if x not in self.voc else x]
        #Embedimiento
        x = self.C.T[x] #x(k) ← C(wt−k)
        #capa oculta
        #a ← tanh(Hx + d)
        a = np.tanh(np.dot(self.U, x) + self.b)[0]
        #salida
        # p_j ← e**(a.U + b_j) 
        # if (direct connections) e**(e**(a.U + b_j) + x.W_j)
        out = np.exp(np.dot(self.W, a) + self.c)[0]
        #Softmax
        # Normalize the probabilities
        self.p = out/out.sum(0)
        return self.p, a

    def plot_words(self, ids):
        '''
        Muestra los embedings utilizando PCA

        Args:
            ids (int): valor númerico de la palabra
        '''
        Z = PCA(2).fit_transform(self.C.T[:-2])
        plt.figure(figsize=(10,6))
        plt.scatter(Z[:,0],Z[:,1], marker='.')
        for label,x,y in zip(ids, Z[:,0], Z[:,1]):
            plt.annotate(label, xy=(x,y), xytext=(-1,1), 
                         textcoords='offset points', 
                         ha='center', va='bottom')
        plt.show()

    def prob_sentence(self, sentence):
        '''
        Obtenemos la probabilidad de la oración

        Args:
            sentence (list): lista de las palabras que componen la oración
        '''
        #Obtenemos los bigramas de la cadena de evaluacion
        bigrams = list(zip(sentence,sentence[1:]))
        p = 1
        #Multiplicamos por las probabilidades de los bigramas dado el modelo
        for gram1, gram2 in bigrams:
            #Obtiene las probabilidades de transición
            try:
                prev_prob = self.forward(gram1)[0]
            except:
                prev_prob = self.forward('<oov>')[0]

            try:
                p *= prev_prob[gram2]
            except:
                p *= prev_prob['<oov>']
                
        return p 

    def get_entropy(self, test_data):
        '''
        Obtenemos la entropia promedio del modelo

        Args:
            test_data (list): conjunto de prueba tokenizado
        '''
        H = 0.0
        # calculamos entropia como el promedio de las probabilidades de cada oración
        for sentence in tqdm(test_data):
            #Probabilidad de la cadena
            p_cad = self.prob_sentence(sentence)
            #Longitud de la cadena
            M = len(sentence)
            #Obtenemos la entropía cruzada de la cadena
            if p_cad != 0:
                H -= (1./M)*(np.log(p_cad)/np.log(2))

        return H/len(test_data)
    
    def test(self, test):
        '''
        Probamos el modelo

        Args:
            test (list): conjunto de prueba tokenizado
        '''
        entropy = self.get_entropy(test)
        perplexity = 2**entropy
        return entropy, perplexity

    def save_embedings(self, path):
        '''
        Guardamos en diccionario (palabra: embeding) en un
        archivo

        Args:
            path (str): ruta para guardar el archivo
        '''
        embedings = {}
        for word in self.voc.keys():
            embedings[word] = self.C.T[self.voc[word]]

        pickle.dump(embedings, open(path, 'wb'))

    def save_model(self, path):
        """
        Para guardar el modelo ya entrenado en un archivo

        Args:
            path (str): ruta para guardar el archivo
        """
        pickle.dump(self, open(path, 'wb'))

Obtenemos modelos entrenados en la práctica 2

In [ ]:
# Comprobamos que se guardó correctamente
bengio_full = pickle.load(open('./modelAll/model.pkl', 'rb'))
bengio_10 = pickle.load(open('./model10%/model.pkl', 'rb'))

Funciones para generar palabras y cadenas

In [ ]:
more = ["wont","_","-","'ve", "'ll", "'t", "'s", "'re", "'", "'m", "'d", "n't", "oh", "hey", "yeah","okay", "mr.", "miss", "mrs."]
stopwords_list = stopwords.words('english') + more

def get_ordered_probs(word, bengio):
    """
    Obtiene las probabilidades ordenadas de mayor a menor

    Args:
        word (str): palabra base
        bengio (Bengio): Objeto de la clase bengio
    """
    probs, a = bengio.forward(word)
    dic_probs = dict(zip(bengio.voc.keys(), probs))
    dic_probs.pop('<oov>')
    return sorted(dic_probs.items(), key=itemgetter(1), reverse=True)

def next_word(string, bengio):
    """
    Dada una palabra genera ls palabra siguiente

    Args:
        string (str): palabra base
        bengio (Bengio): Objeto de la clase bengio
    """
    #Obtener la última palabra en la historia
    last_w = string.split()[-1]
    #Obtener una palabra en base a la distribución
    selection = np.random.choice(range(4), 1, p=None)[0]
    max_w = get_ordered_probs(last_w, bengio)[selection]   
    return max_w[0]

def get_query_clean(text):
    '''
    Genera los tokens de una cadena y los limpia
    (quita símbolos raros y stopwords)
    
    Args:
        text (str): cadena
    '''
    tokens = word_tokenize(text)
    clean = []
    pattern = r'[^a-z0-9\s]'
    for w in tokens:
        #quita stopwords y convierte a minúsculas
        w = re.sub(pattern,'', w.lower())
        if w not in stopwords_list and w != '':
            if  w == "na": #Para juntar gon na, wan na, etc.
                clean[-1] += w
            else:
                clean.append(w)
    return clean

def generate(string, bengio):
    '''
    Genera la cadena que le sigue a la cadena de inicio 
    
    Args:
        string (str): cadena base
        bengio (Bengio): Objeto de la clase bengio
    '''
    # limpiamos entrada
    string = ' '.join(get_query_clean(string))
    #Guarda la palabra predicha
    w = ''
    #Guarda la cadena que se ha generado
    str_gen = string
    #El método se detiene al ver <EOS>
    t = 0
    while w != '<EOS>':
        #Predice la siguiente palabra
        w = next_word(str_gen, bengio)
        #Agrega esa palabra a ala cadena
        str_gen += ' ' + w
        t += 1
        if t == 10:
            w = '<EOS>'
    
    str_gen = str_gen.replace('<BOS>', '. ')
    #Regresa la cadena si el símbolo EOS
    return str_gen[:len(str_gen)]

Probamos los 2 modelos que tenemos:
- bengio_full se entrenó con 100 iteraciones utilizando todo el corpus
- bengio_10 se entrenó con 10 iteraciones utilizando igual todo el corpus

In [ ]:
generate('Stephen Strange', bengio_full)

In [ ]:
generate('Stephen Strange', bengio_10)

Generando oraciones para ser evaluadas

In [ ]:
test = pickle.load(open('./pickles/datasets/test.pkl','rb'))

print('Número de cadenas test:',len(test))
print(test[:3])

Probamos en el conjunto de test

In [ ]:
predict_bengio = []

import math

for sentence in tqdm(test):
    sentence_splited = get_query_clean(sentence)
    length = len(sentence_splited)
    half_index = math.ceil(length / 2)
    half = ' '.join(sentence_splited[:half_index])

    predict_bengio.append('-' if len(half) == 0 else generate(half, bengio_full))

predict_bengio[:3]

In [ ]:
pickle.dump(predict_bengio, open('./pickles/predict/bengio.pkl', 'wb'))